# Classification de Tickets de Support Client

Ce notebook présente un pipeline complet pour la classification automatique des types de tickets de support client. Nous utilisons un modèle de machine learning basé sur TF-IDF et un réseau de neurones (MLPClassifier) pour prédire le type d'un ticket à partir de son sujet et corps.

## Objectifs
- Charger et préparer un dataset de tickets de support depuis Hugging Face.
- Prétraiter les données textuelles.
- Entraîner un modèle de classification avec un réseau de neurones.
- Évaluer les performances du modèle.
- Sauvegarder et recharger le modèle pour des prédictions sur de nouveaux tickets.

## Technologies utilisées
- **Datasets** : Pour charger le dataset Hugging Face.
- **Pandas** : Pour la manipulation des données.
- **Scikit-learn** : Pour le preprocessing (TF-IDF), le modèle (MLPClassifier), et l'évaluation.
- **Joblib** : Pour la sauvegarde et le chargement du modèle.

## Étapes du pipeline
1. Installation et imports des bibliothèques.
2. Chargement du dataset.
3. Préparation des données (DataFrame, création du texte combiné).
4. Encodage des labels et division train/test.
5. Définition du pipeline TF-IDF + MLPClassifier.
6. Entraînement du modèle.
7. Évaluation sur le jeu de test.
8. Sauvegarde du modèle.
9. Exemple de prédiction sur un nouveau ticket.

## Installation et Imports

Dans cette section, nous installons et importons toutes les bibliothèques nécessaires pour ce projet. Assurez-vous d'avoir un environnement Python configuré avec les dépendances listées dans `requirements.txt`.

- `datasets` : Bibliothèque Hugging Face pour charger des datasets.
- `pandas` : Pour la manipulation de données tabulaires.
- `sklearn` : Modules pour le machine learning (modèle, preprocessing, évaluation).
- `joblib` : Pour la sérialisation des modèles.

In [ ]:
# Import des bibliothèques nécessaires
from datasets import load_dataset  # Pour charger le dataset Hugging Face
import pandas as pd  # Manipulation de données

# Imports pour le machine learning
from sklearn.model_selection import train_test_split  # Division des données
from sklearn.feature_extraction.text import TfidfVectorizer  # Vectorisation TF-IDF
from sklearn.neural_network import MLPClassifier  # Modèle de réseau de neurones (MLP)
from sklearn.pipeline import Pipeline  # Pipeline pour combiner les étapes
from sklearn.metrics import classification_report, confusion_matrix  # Évaluation
import joblib  # Sauvegarde du modèle

## Chargement du Dataset Hugging Face

Nous utilisons le dataset "Tobi-Bueck/customer-support-tickets" disponible sur Hugging Face. Ce dataset contient des tickets de support client avec des sujets, corps et types de tickets.

- **Source** : Hugging Face Datasets.
- **Contenu** : Tickets avec colonnes 'subject', 'body', 'type'.
- **Utilisation** : Charger le split 'train' pour l'entraînement.

In [ ]:
# Charger le dataset customer-support-tickets depuis Hugging Face
dataset = load_dataset("Tobi-Bueck/customer-support-tickets")
dataset  # Afficher la structure du dataset

c:\Users\achou\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\achou\.cache\huggingface\hub\datasets--Tobi-Bueck--customer-support-tickets. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 61765/61765 [00:01<00:00, 37756.69 

DatasetDict({
    train: Dataset({
        features: ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8'],
        num_rows: 61765
    })
})

## Passage en DataFrame et Création du Texte

Nous convertissons le dataset Hugging Face en DataFrame Pandas pour une manipulation plus facile. Nous combinons le sujet et le corps du ticket en une seule colonne 'text' pour l'entrée du modèle.

- **Colonnes gardées** : 'subject', 'body', 'type'.
- **Nouvelle colonne** : 'text' = subject + " " + body.
- **Nettoyage** : Suppression des lignes avec valeurs manquantes.

In [ ]:
# On suppose qu'on utilise le split "train"
df = dataset["train"].to_pandas()

# On garde les colonnes qui nous intéressent
# subject + body = texte d'entrée, type = label
df = df[["subject", "body", "type"]].dropna()

# Créer une colonne 'text' = subject + body
df["text"] = df["subject"].astype(str) + " " + df["body"].astype(str)

# Afficher un aperçu
df.head()

,subject,body,type,text
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Incident,Wesentlicher Sicherheitsvorfall Sehr geehrtes ...
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...",Incident,"Account Disruption Dear Customer Support Team,..."
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Request,Query About Smart Home System Integration Feat...
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",Request,Inquiry Regarding Invoice Details Dear Custome...
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Problem,Question About Marketing Agency Software Compa...


## Encodage des Labels et Split Train/Test

Les labels (types de tickets) sont des chaînes de caractères. Nous les encodons en entiers pour le modèle. Puis, nous divisons les données en ensembles d'entraînement (80%) et de test (20%).

- **Encodage** : Création de dictionnaires label2id et id2label.
- **Stratification** : Pour maintenir la distribution des classes.
- **Random state** : Pour la reproductibilité.

In [ ]:
# Encodage des labels (type) en entiers
label2id = {label: idx for idx, label in enumerate(sorted(df["type"].unique()))}
id2label = {idx: label for label, idx in label2id.items()}

df["label"] = df["type"].map(label2id)

X = df["text"].values  # Textes d'entrée
y = df["label"].values  # Labels encodés

# Split train / test (par ex. 80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

len(X_train), len(X_test)  # Afficher les tailles

(34628, 8658)

## Définition du Pipeline TF-IDF + MLPClassifier

Nous définissons un pipeline Scikit-learn qui combine la vectorisation TF-IDF avec un classificateur de réseau de neurones (MLP : Multi-Layer Perceptron).

- **TF-IDF** : Transforme le texte en vecteurs numériques, en pondérant les termes par leur fréquence inverse dans le corpus.
- **Paramètres TF-IDF** : max_features=20000, ngram_range=(1,2), stop_words="english".
- **MLPClassifier** : Réseau de neurones feedforward avec couches cachées, adapté à la classification multi-classe. Paramètres : hidden_layer_sizes=(100, 50) pour deux couches cachées, max_iter=1000 pour convergence, random_state pour reproductibilité.

In [ ]:
# Pipeline : TF-IDF (texte) -> MLPClassifier (réseau de neurones)
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=20000,   # pour limiter la taille, ajustable
        ngram_range=(1, 2),   # unigrams + bigrams
        stop_words="english"  # ou None si tu veux garder tout
    )),
    ("clf", MLPClassifier(
        hidden_layer_sizes=(100, 50),  # Deux couches cachées : 100 neurones puis 50
        max_iter=1000,                 # Nombre max d'itérations pour convergence
        random_state=42,               # Pour reproductibilité
        early_stopping=True,           # Arrêt précoce pour éviter le surapprentissage
        n_iter_no_change=10            # Tolérance pour early stopping
    ))
])

## Entraînement du Modèle

Nous entraînons le pipeline sur les données d'entraînement. Cette étape peut prendre quelques minutes selon la taille du dataset et les ressources disponibles.

In [ ]:
# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

c:\Users\achou\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=20000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('clf',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    n_jobs=-1))])

## Évaluation sur le Jeu de Test

Nous évaluons les performances du modèle sur les données de test non vues pendant l'entraînement. Les métriques incluent la précision, le rappel, le F1-score et la matrice de confusion.

In [ ]:
# Prédire sur le jeu de test
y_pred = model.predict(X_test)

print("Rapport de classification :")
print(classification_report(y_test, y_pred, target_names=[id2label[i] for i in sorted(id2label.keys())]))

print("Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

Rapport de classification :
              precision    recall  f1-score   support

      Change       0.99      0.88      0.93       878
    Incident       0.77      0.90      0.83      3477
     Problem       0.72      0.50      0.59      1815
     Request       0.97      0.99      0.98      2488

    accuracy                           0.84      8658
   macro avg       0.86      0.82      0.83      8658
weighted avg       0.84      0.84      0.83      8658

Matrice de confusion :
[[ 776   15    8   79]
 [   2 3121  349    5]
 [   1  897  914    3]
 [   7    5    1 2475]]


## Sauvegarde du Modèle Entraîné

Nous sauvegardons le pipeline complet, ainsi que les mappings label2id et id2label, dans un fichier joblib pour une utilisation future.

In [ ]:
# Sauvegarder tout le pipeline (TF-IDF + régression logistique)
joblib.dump({
    "pipeline": model,
    "label2id": label2id,
    "id2label": id2label
}, "ticket_type_classifier.joblib")

"Modèle sauvegardé dans ticket_type_classifier.joblib"

'Modèle sauvegardé dans ticket_type_classifier.joblib'

## Rechargement du Modèle et Prédiction sur un Nouvel Exemple

Nous rechargeons le modèle sauvegardé et testons une prédiction sur un exemple de ticket fictif pour démontrer l'utilisation en production.

In [ ]:
# Recharger le modèle sauvegardé
saved = joblib.load("ticket_type_classifier.joblib")
loaded_model = saved["pipeline"]
id2label = saved["id2label"]

# Exemple de ticket
nouveau_subject = "Problem with my invoice"
nouveau_body = "I was charged twice this month, please fix this issue."
nouveau_text = nouveau_subject + " " + nouveau_body

# Prédiction
pred_label_id = loaded_model.predict([nouveau_text])[0]
pred_label = id2label[pred_label_id]

print("Texte :", nouveau_text)
print("Type prédit :", pred_label)

Texte : Problem with my invoice I was charged twice this month, please fix this issue.
Type prédit : Problem


## Conclusion

Ce notebook démontre un pipeline complet pour la classification de tickets de support client utilisant TF-IDF et un réseau de neurones (MLPClassifier). Le modèle peut être utilisé pour automatiser le tri des tickets entrants dans un système de support.

### Améliorations Possibles
- **Modèles plus avancés** : Essayer des transformers comme BERT pour de meilleures performances sur le texte.
- **Preprocessing** : Ajouter du nettoyage de texte (lemmatisation, suppression de ponctuation).
- **Évaluation** : Utiliser la validation croisée pour une évaluation plus robuste.
- **Déploiement** : Intégrer le modèle dans une API Flask ou FastAPI pour un usage en production.

### Ressources
- Dataset : [Hugging Face - customer-support-tickets](https://huggingface.co/datasets/Tobi-Bueck/customer-support-tickets)
- Scikit-learn Documentation : [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)